## Docs
* https://dev.mysql.com/doc/
* https://docs.sqlalchemy.org/en/20/core/engines.html#mysql
* https://jupyter-tutorial.readthedocs.io/en/stable/data-processing/postgresql/ipython-sql.html
* [6.1 Tutorial: Raise Employee's Salary Using a Buffered Cursor](https://dev.mysql.com/doc/connector-python/en/connector-python-tutorial-cursorbuffered.html)

## Ref
* https://towardsdatascience.com/jupyter-magics-with-sql-921370099589

In [1]:
import os
import pandas as pd

import mysql.connector
from dotenv import load_dotenv
from mysql.connector import Error
from sqlalchemy.engine import create_engine

## enviroment

In [2]:
load_dotenv()

_MYSQL_ROOT_USER = os.getenv('MYSQL_ROOT_USER')
_MYSQL_USER = os.getenv('MYSQL_USER')
_MYSQL_ROOT_PASSWORD = os.getenv('MYSQL_ROOT_PASSWORD')
_MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
_MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')
# _MYSQL_USER = os.getenv('MYSQL_USER')
_DB_PORT = os.getenv('DB_PORT')
_DATABASE = os.getenv('DATABASE')

## execution pattern

### Type1: Pandas

In [3]:
URL =f"mysql+pymysql://{_MYSQL_USER}:{_MYSQL_PASSWORD}@{_MYSQL_DATABASE}:{_DB_PORT}/{_DATABASE}"
# URL =f"mysql+pymysql://{_MYSQL_ROOT_USER}:{_MYSQL_ROOT_PASSWORD}@{_MYSQL_DATABASE}:{_DB_PORT}/{_DATABASE}"
engine = create_engine(URL)

In [4]:
df = pd.read_sql('SELECT * FROM Album limit 10', engine)

In [5]:
df.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### Type2: magic CMD

In [6]:
# init
%load_ext sql
%config SqlMagic.displaycon=False 
%config SqlMagic.autopandas=True

host_ = '172.25.0.2'
%sql mysql+pymysql://{_MYSQL_USER}:{_MYSQL_PASSWORD}@{host_}/{_DATABASE}

In [7]:
%%sql result_set << 
SELECT * FROM Album ORDER BY AlbumId DESC LIMIT 5

5 rows affected.
Returning data to local variable result_set


In [8]:
result_set

,AlbumId,Title,ArtistId
0,347,Koyaanisqatsi (Soundtrack from the Motion Pict...,275
1,346,Mozart: Chamber Music,274
2,345,Monteverdi: L'Orfeo,273
3,344,Schubert: The Late String Quartets & String Qu...,272
4,343,Respighi:Pines of Rome,226


---

## 🗒 NOTE

### コンテナDB の IP アドレスを検索する
※ コンテナ 再度立ち上げると IP 割り当てかわっている
```bash
# ex:
$ docker inspect {CONTAINER ID} | grep IPAddress
            "SecondaryIPAddresses": null,
            "IPAddress": "",
                    "IPAddress": "172.23.0.2",
```                    